In [279]:
import numpy as np
import pandas as pd
import operator
import pickle
import shelve
import re
from collections import Counter, defaultdict, OrderedDict, deque
import csv
import rdflib
from rdflib.namespace import OWL, RDF, RDFS
from rdflib import URIRef
import os
import string
import math

In [280]:
#####################################################
################ Initialization #####################
#####################################################

#ground truth metadata file
labeledFile = 'ebu3b_sensor_types_location.csv'
with open(labeledFile, 'rb') as fp:
    #Import list of points to a data frame
    #Pandas data frame allows you to store records and manipulate them: http://pandas.pydata.org/
    points_list = pd.DataFrame.from_csv(fp)
    points_list = points_list.set_index(keys='Unique Identifier')
    points_list.index.str.strip()
    equip_subsystem = points_list['Equipment Subsystems'] #saving to preserve formatting
    points_list = points_list.replace(r'\s+|-', '_', regex=True)
    
#Import lookup table that maps EBU3B descriptions to Brick tagsets
desc_tagset_map = pd.read_csv('ebu3b_description_to_tagset.csv')
desc_tagset_map = desc_tagset_map.replace(r'\s+','_',regex=True).set_index('BACnet Description').dropna()

In [281]:
#####################################################
######## Initialize RDFlib graph and namespaces #####
#####################################################

#Namespaces
#Note: The URLs are fake
BRICK = rdflib.Namespace('http://buildsys.org/ontologies/Brick#')
BRICKFRAME = rdflib.Namespace('http://buildsys.org/ontologies/BrickFrame#')
EBU3B = rdflib.Namespace('http://ucsd.edu/building/ontology/ebu3b#')
#RDF, RDFS and OWL have already been imported in the library initializations

#Initiate graph from base ttl file
building_graph = rdflib.Graph()
brick_graph = rdflib.Graph()
building_graph.bind('ebu3b', EBU3B)
building_graph.bind('brick', BRICK)
building_graph.bind('brickframe', BRICKFRAME)
building_graph.bind('owl', OWL)
brick_graph.bind('brick', BRICK)
brick_graph.bind('brickframe',BRICKFRAME)
brick_graph.parse('../../BuildingSchema/Brick.ttl', format='turtle')
#building_graph.parse('../../BuildingSchema/Brick_Protege.owl', format='turtle')
#brick_graph.serialize(destination='brick_rdflib.ttl', format='turtle')

<Graph identifier=N076ccf19055a4f0a8f43f6bc53d9a214 (<class 'rdflib.graph.Graph'>)>

In [282]:
#Adding Location information
#Convert location to title case
points_list['Location'] = points_list['Location'].apply((lambda x: x.title() if isinstance(x, basestring) else x))
location_list = points_list.reset_index().Location.drop_duplicates().dropna()
location_list = list(location_list)
location_list = [x for x in location_list if 'xxx' not in x]

#Add building and floors
building_graph.add((EBU3B.EBU3B, RDF.type, BRICK.Building))
building_graph.add((EBU3B.EBU3B_Basement, RDF.type, BRICK.Floor))
building_graph.add((EBU3B.EBU3B_Floor_1, RDF.type, BRICK.Floor))
building_graph.add((EBU3B.EBU3B_Floor_2, RDF.type, BRICK.Floor))
building_graph.add((EBU3B.EBU3B_Floor_3, RDF.type, BRICK.Floor))
building_graph.add((EBU3B.EBU3B_Floor_4, RDF.type, BRICK.Floor))
building_graph.add((EBU3B.EBU3B_Basement, BRICKFRAME.isPartOf, EBU3B.EBU3B))
building_graph.add((EBU3B.EBU3B_Floor_1, BRICKFRAME.isPartOf, EBU3B.EBU3B))
building_graph.add((EBU3B.EBU3B_Floor_2, BRICKFRAME.isPartOf, EBU3B.EBU3B))
building_graph.add((EBU3B.EBU3B_Floor_3, BRICKFRAME.isPartOf, EBU3B.EBU3B))
building_graph.add((EBU3B.EBU3B_Floor_4, BRICKFRAME.isPartOf, EBU3B.EBU3B))

for room in location_list:
    #convert to title case
    #room = room.title()
    room_label = EBU3B[room]
    zone_label = EBU3B["EBU3B_HVAC_Zone_"+room]
    #add room as HVAC Zone
    building_graph.add((zone_label, RDF.type, BRICK.HVAC_Zone))
    #get floor
    floor = "EBU3B_Floor_" + room[3] if "Rm" in room else None
    if floor:
        floor = "EBU3B_Basement" if "Floor_B" in floor else floor
        building_graph.add((zone_label, BRICKFRAME.isPartOf, EBU3B[floor]))
    #TODO: Add rooms contained in a zone
    #building_graph.add((room_label, RDF.type, BRICK.Room))
    #building_graph.add((room_label, BRICKFRAME.isPartOf, zone_label))

In [283]:
#Load all equipment to building graph
#Create equipment id
points_list['equip_id'] = points_list['Building'] + '_' + equip_subsystem.replace(r' -> ', '_', regex=True)
points_list.equip_id = points_list.equip_id.replace(r'\s+', '_', regex=True)

#Get equipment related terms
equip_list = points_list[['Building','Equipment Type','Equipment Ref','Location','equip_id']]
equip_list['equip_subsystem'] = equip_subsystem 

#print equip_list.equip_id.head()
#print equip_list[equip_list.equip_id.str.contains('Hot_Water_System')].equip_id
    
#Drop duplicates to get list of equipment only
equip_list = equip_list.set_index('equip_id').drop_duplicates()

#Iterate through list and add to schema
for equip_id, equip_metadata in equip_list.iterrows():
    #check for nan and empty strings
    if equip_id != equip_id or equip_id.strip() == '':
        continue
    #Add EBU3B prefix
    equip_label = EBU3B[equip_id]
    #Add to schema
    building_graph.add((equip_label, RDF.type, OWL.NamedIndividual))
    
    #Add equipment type
    equip_type = str(equip_metadata['Equipment Type'])
    if equip_type == 'nan': #skip blanks
        continue
    if 'CRAC' in equip_type: #CRAC formatting
        equip_type = 'CRAC'
    equip_type = BRICK[equip_type]
    if (equip_type, None, None) in brick_graph:
        #Add equip type to schema
        building_graph.add((equip_label, RDF.type, equip_type))
    else:
        print "Could not identify equipment type in schema:", equip_type
        building_graph.add((equip_label, RDF.type, BRICK.Equipment))
        
    #All the VAVs are fed by AHU_1, and VAVs feed their corresponding HVAC Zones
    if "VAV" in equip_id and not(any(x in equip_id for x in ["Thermostat","Damper","Valve","Fan","Unknown"])):
        building_graph.add((EBU3B.EBU3B_AHU_1, BRICKFRAME.feeds, equip_label))
        zone_id = equip_id.replace("VAV", "HVAC_Zone")
        building_graph.add((equip_label, BRICKFRAME.feeds, EBU3B[zone_id]))

/Users/bbalaji/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [284]:
#Add equipment isPartOf relationships
for equip_id, equip_metadata in equip_list.iterrows():
    #check for nan and empty strings
    if equip_id != equip_id or equip_id.strip() == '':
        continue
    #Parse the equipment hierarchy from CSV file
    equip_parts = equip_metadata['equip_subsystem'].split(' -> ')
    equip_hier = []
    for i,part in enumerate(equip_parts):
        part = '_'.join(part.split())
        part_ref = '_'.join(equip_parts[i+1].split()) if i+1 < len(equip_parts) else None
        if (BRICK[part],None, None) in brick_graph:
            if (BRICK[part_ref],None,None) in brick_graph or part_ref == None: 
                part_id = part
                equip_ref = equip_id.split(part_id)[0] + part_id
                equip_hier.append(equip_ref)
            else:
                part_id = part + '_' + part_ref
                equip_ref = equip_id.split(part_id)[0] + part_id
                equip_hier.append(equip_ref)
    equip_hier.reverse()
    #print equip_hier
    for i,equip_id in enumerate(equip_hier):
        if i+1 < len(equip_hier):
            building_graph.add((EBU3B[equip_id], BRICKFRAME.isPartOf, EBU3B[equip_hier[i+1]]))
            #print equip_id, "isPartOf", equip_hier[i+1]
        

In [285]:
#Load all points to the building instance
for point_id, point_metadata in points_list.iterrows():
    #Create point label
    point_label = re.sub('\.+','_',point_metadata['Johnson Controls Name'])
    #Add building prefix
    point_label = EBU3B[point_label]
    #Add to Schema
    building_graph.add((point_label, RDF.type, OWL.NamedIndividual))

    #Adding Point Type
    #Look up the schema tagset
    desc = point_metadata['BACnet Description']
    if desc in desc_tagset_map.index and str(desc) != 'nan':
        point_type = desc_tagset_map.loc[desc]['Schema Label']
    else:
        continue
    if isinstance(point_type, pd.Series):
        point_type = point_type[0]
    if str(point_type) == 'nan':
        continue
    #Get the first match, capitalize the first letter of each word and concatenate by '_'
    #print point_type
    point_type = '_'.join([word[0].upper() + word[1:] for word in point_type.split('_')])
    #Take care of Synonyms
    if 'Supply Air' in point_type: 
        point_type = point_type.replace('Supply Air','Discharge Air')
    #Concatenate with brick prefix
    point_type = BRICK[point_type]
    #print point_type
    #Check if point type exists in the schema
    if (point_type, None, None) in brick_graph:
        #Add point type to schema
        building_graph.add((point_label, RDF.type, point_type))
    else:
        print "Could not identify point type in schema:", point_type 
        building_graph.add((point_label, RDF.type, BRICK.Point))
        
    #Add isPointOf relationship
    equip_id = point_metadata['equip_id']
    if equip_id != equip_id or equip_id.strip() == '':
        continue
    equip_label = EBU3B[equip_id]
    #Add to schema
    building_graph.add((point_label, BRICKFRAME.isPointOf, equip_label))
    #Add transitive isPointOf for VAV and AHU #can be removed, depending on app requirements
    if "AHU" in equip_id:
        equip_id = '_'.join(equip_id.split('_')[:3])
        building_graph.add((point_label, BRICKFRAME.isPointOf, EBU3B[equip_id]))
    elif "VAV" in equip_id and "unknown" not in equip_id.lower():
        equip_id = '_'.join(equip_id.split('_')[:4])
        building_graph.add((point_label, BRICKFRAME.isPointOf, EBU3B[equip_id]))
    #Add thermostat points to the zone
    equip_id = point_metadata['equip_id']
    if "Thermostat" in equip_id and "Unknown" not in equip_id:
        zone_label = EBU3B["EBU3B_HVAC_Zone_"+point_metadata["Location"]]
        building_graph.add((point_label, BRICKFRAME.isPointOf, zone_label))
        #print point_label, 'isPointOf', zone_label

Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://buildsys.org/ontologies/Brick#Unknown
Could not identify point type in schema: http://builds

In [286]:
# Store output as .ttl
building_graph.serialize(destination='ebu3b_brick.ttl', format='turtle')
# Search for "EBU3B" in the output file, you'll find the instances we created.

In [ ]:
#####################################################
######## Add triples and serialize them #############
#####################################################

#### Example for Point ####

###Adding Label####
#Read the point metadata by its unique identifier
point = points_list.loc['505_0_3001627']
#Create a label for the point
point_label = re.sub('\.+','_',point['Johnson Controls Name'])
#Add building prefix
point_label = EBU3B[point_label]
#print point_label
#Add to Schema
building_graph.add((point_label, RDF.type, OWL.NamedIndividual))

####Adding Point Type###
#Look up the schema tagset
point_type = desc_tagset_map.loc[point['BACnet Description']]['Schema Label']
#Get the first match, capitalize the first letter of each word and concatenate by '_'
point_type = '_'.join(point_type[0].title().split())
#Concatenate with brick prefix
point_type = BRICK[point_type]
#print point_type
#Check if point type exists in the schema
if (point_type, None, None) in brick_graph:
    #Add point type to schema
    building_graph.add((point_label, RDF.type, point_type))
else:
    print "Could not identify point type in schema:", point_type 

In [ ]:
###Example for a VAV instance ####
#Continuing with the point information above

##Create Equipment Label##
equip_id = point['Building'] + '_' + point['Equipment Type'] + '_' + point['Location']
#replaces spaces with '_'
equip_id = '_'.join(equip_id.split())
#add building prefix
equip_id = EBU3B[equip_id]
#print equip_id
#add to schema instance
building_graph.add((equip_id, RDF.type, OWL.NamedIndividual))

##Add equipment type##
equip_type = point['Equipment Type']
#add brick prefix
equip_type = BRICK[equip_type]
#print equip_type
#Check if equip type exists in the schema
if (equip_type, None, None) in brick_graph:
    #Add equip type to schema
    building_graph.add((equip_id, RDF.type, equip_type))
else:
    print "Could not identify equipment type in schema:", equip_type 
    
##Add relationship between point and equipment ##
building_graph.add((equip_id, BRICK.hasPoint, point_label))

In [ ]:
#TODO
#Iterate through all the points in the building and map them to schema
for point_id, point_metadata in points_list.iterrows():
    print point_metadata
    break

In [ ]:
#### For Graph Traversal #####
#For traversing the Brick or EBU3B graph, use the "Resource" part of RDFlib
#Tutorial here: http://rdflib.readthedocs.io/en/stable/apidocs/rdflib.html#module-rdflib.resource
#Example subclass traversal is shown below
valve = rdflib.resource.Resource(g, BRICK.Valve)
subclasses = list(valve.transitive_subjects(RDFS.subClassOf))
[c.qname() for c in subclasses]